# Relleno de faltantes

Carga base de datos por estación

[Guia de datos faltantes](https://www.kaggle.com/parulpandey/a-guide-to-handling-missing-values-in-python)


## Funcion para obtener la tabla de faltantes

In [33]:
# Carga paquetes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from keras.preprocessing.sequence import TimeseriesGenerator


In [34]:
#Missing Values
# credit: https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction. 
# One of the best notebooks on getting started with a ML problem.

def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns


# Convierte WSP y WDR en X Y para promediar el viento

pasar WDR de grados a radianes _WDR=WDR*np.pi/180_

obtener la conversión a coordenadas polares a cartesianas
_X=WSP*np.cos(WDR)_

_Y=WSP*np.sin(WDR)_

promediar los registros de X y Y y convertir el promedio a WSP y WDR

_WSP=raiz(X**2+Y**2)_

_WDR=np.arctan(Y/X)_

dependiendo del cuadrante hay que corregir WDR para que conicida el Norte con 0 o 360°, Este con 90°, Sur con 180° y Oeste con 270°  

In [4]:
# Activa el repositorio
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [35]:
#establece la ruta
ruta="/content/drive/MyDrive/AIre/BD/"

In [36]:
# Lee las rutas de los archivos
paths=glob.glob(ruta + '*_2015_2020.csv')
paths

['/content/drive/MyDrive/AIre/BD/CCA_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/HGM_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/MER_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/MON_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/NEZ_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/PED_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/SFE_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/TLA_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/XAL_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/GAM_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/AJM_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/AJM2_2015_2020.csv',
 '/content/drive/MyDrive/AIre/BD/CCA2_2015_2020.csv']

In [37]:
# Corrobora el nombre de los sitios de monitoreo
for dff in paths:
  print(dff[-17:-14])


CCA
HGM
MER
MON
NEZ
PED
SFE
TLA
XAL
GAM
AJM
JM2
CA2


# Análisis de los sitios AJM, MER y XAL

In [40]:
# Proceso para cada estación

for dff in paths:  
  # Genera una serie horaria de inicio a fin
  dates=pd.date_range(start='2015-1-1', end='2019-9-30', freq='H')
  dates=pd.DataFrame(dates)
  dates.columns=['date2']

  # Asigna el nombre del sitio
  sitio=dff[-17:-14]
  dff2='/content/drive/MyDrive/AIre/BD/' + sitio + '2_2015_2020.csv'
  # Lee el archivo, cambia formato de la fecha, genera mes y elimina columnas sin datos
  df=pd.read_csv(dff)
  print(df.head(3))
  df.date2 = pd.to_datetime(df.date2, format="%Y-%m-%d %H:%M")
  df['Month']=df['date2'].dt.month
  df=df.set_index('date2')
  df.drop(columns=df.columns[df.isnull().sum()==df.shape[0]],inplace=True,axis=1)
  param=df.columns
  dates[param]=np.NaN
  dates=pd.DataFrame(dates)
  dates=dates.set_index('date2')
  df=pd.DataFrame(df)
  df=pd.merge(df, dates, on='date2',indicator='union')
  df.drop(columns=df.columns[df.isnull().sum()==df.shape[0]],inplace=True,axis=1)
  df.columns=[col.replace("_x","") for col in df.columns ]
  df.drop(columns=['union'],inplace=True,axis=1)
  df=df.reset_index()
  # Respalda la base con fechas completas
  df.to_csv(dff2, index=False)
  print("********************************************************************************************************************* ")
  print("Los parámetros que mide la estación {} son: {}:".format(sitio, df.columns))
  print(" ")
  df=df.set_index(df['date2'])

  # Conversión de WSP y WDR a cartesianas (otra opción es con import math math.sin, math.cos, math.radians)
  if df.columns.isin(["WSP"]).sum()>0:
    wsp=df['WSP']
    wdr=df['WDR']
    df['X']=wsp*np.sin(wdr*np.pi/180)
    df['Y']=wsp*np.cos(wdr*np.pi/180)

  # visualiza el comportamiento de la base original
  print("********************************************************************************************************************* ")
  print("Comportamiento de la base original")
  print(" ")

  df.plot(subplots=True, figsize=(18,25))
  plt.savefig(ruta + "Sitios/" + sitio + "_sinrell.png", bbox_inches='tight')
  plt.show()

  # Despliega la tabla de datos faltantes y genera la lista de parametros a rellenar
  faltantes=missing_values_table(df)
  param=faltantes.index
  print("********************************************************************************************************************* ")
  print('Los faltantes van de {}% a {}%'.format(faltantes['% of Total Values'].min(),faltantes['% of Total Values'].max()))
  print(" ")

  # Explora el comportamiento de los faltantes
  import missingno as msno
  msno.bar(df[param])
  msno.matrix(df[param])
  sorted = df[param].sort_values('PM2.5') # Ordena por la varable de interés (PM2.5)
  msno.matrix(sorted)
  msno.heatmap(df[param]).get_figure().savefig(ruta + "Sitios/" + sitio + "_sinrell_heatmap.png", bbox_inches='tight')
  msno.dendrogram(df[param]).get_figure().savefig(ruta + "/Sitios/" + sitio + "_sinrell_dendr.png", bbox_inches='tight')
  print("********************************************************************************************************************* ")
  print("El comportamiento de los datos faltantes no es aleatorio ya que muestra patrones de asociación entre columnas, por un lado las partículas (PM10, PM2.5, PMCO), por otro los gases (NO, NO2, NOX, CO, SO2, O3), otro más entre los meteorológicos (TMP, HR) y (WSP, WDR), y al final PA.")
  print(" ")

  # Relleno de faltantes por hora con la media
  df_a0=df.copy()
  for h in df['Hour'].unique():
    h_index=df[df['Hour']==h].index.tolist()
    df_a0.loc[h_index,:]=df.loc[h_index,:].fillna(df.loc[h_index,:].mean())

  # Convertir X Y a WSP y WDR en df_a0
  if df.columns.isin(["WSP"]).sum()>0:
    df_a0['WSP']=round(np.sqrt(df_a0['X']**2 + df_a0['X']**2),0)
    x0y0=df_a0[(df_a0['X']>=0)&(df_a0['Y']>=0)].index #Cuadrante I: 0 a 90°
    xy=df_a0[(df_a0['X']>=0)&(df_a0['Y']<0)].index #Cuadrante II: 90 a 180°
    x0y=df_a0[(df_a0['X']<0)&(df_a0['Y']<0)].index #Cuadrante III: 180 a 270°
    xy0=df_a0[(df_a0['X']<0)&(df_a0['Y']>0)].index #CuadranteIV: 270 a 360°
    df_a0.loc[x0y0,"WDR"]=(round(90-((np.arctan2(df_a0.loc[x0y0,'Y'],df_a0.loc[x0y0,'X']))*180/np.pi),0)) #arctan2
    df_a0.loc[xy,"WDR"]=(round(90-((np.arctan2(df_a0.loc[xy,'Y'],df_a0.loc[xy,'X']))*180/np.pi),0)) #arctan2
    df_a0.loc[x0y,"WDR"]=round(270-((np.arctan(df_a0.loc[x0y,'Y']/df_a0.loc[x0y,'X']))*180/np.pi),0) #arctan
    df_a0.loc[xy0,"WDR"]=round(270-((np.arctan(df_a0.loc[xy0,'Y']/df_a0.loc[xy0,'X']))*180/np.pi),0) #arctan


  # Corrobora que no falta ningun día
  print("********************************************************************************************************************* ")
  print("Faltantes diarios: ",missing_values_table(df_a0.resample('D').mean()))
  print(" ")

  # Respalda base con relleno por hora
  df_a0.to_csv(ruta + "Sitios/" + sitio + "_rell00.csv", index=False)         

  # visualiza el comportamiento de la base con relleno de faltantes por hora
  print("********************************************************************************************************************* ")
  print("Comportamiento de la base con relleno de faltantes")
  print(" ")

  df_a0.plot(subplots=True, figsize=(18,25))
  plt.savefig(ruta + "Sitios/" + sitio + "_rell00.png", bbox_inches='tight')
  plt.show()
  faltantes=missing_values_table(df_a0)
  print("********************************************************************************************************************* ")
  print('Los faltantes van de {}% a {}%'.format(faltantes['% of Total Values'].min(),faltantes['% of Total Values'].max()))
  print(faltantes)
  print(" ")

  print("********************************************************************************************************************* ")
  print("Columnas en la base de datos final: ",df_a0.columns)

  # # Relleno de faltantes por año-mes-hora con la media
  # df_a=pd.DataFrame()
  # for y in df['year'].unique():  
  #   m=0
  #   for m in df['Month'].unique():
  #     df_gp=df[(df['year']==y)&(df['Month']==m)]
  #     df_gp_k=df_gp.copy()
  #     h=0
  #     for h in df_gp['Hour'].unique():
  #         h_index=df_gp[df_gp['Hour']==h].index.tolist()
  #         df_gp_k.loc[h_index,:]=df_gp.loc[h_index,:].fillna(df_gp.loc[h_index,:].mean()) 
  #         df_a=pd.concat([df_a,df_gp_k],axis=0)

  # # Convertir X Y a WSP y WDR en df_a
  # if df.columns.isin(["WSP"]).sum()>0:
  #   df_a['WSP']=round(np.sqrt(df_a['X']**2 + df_a['X']**2),0)
  #   x0y0=df_a[(df_a['X']>=0)&(df_a['Y']>=0)].index #Cuadrante I: 0 a 90°
  #   xy=df_a[(df_a['X']>=0)&(df_a['Y']<0)].index #Cuadrante II: 90 a 180°
  #   x0y=df_a[(df_a['X']<0)&(df_a['Y']<0)].index #Cuadrante III: 180 a 270°
  #   xy0=df_a[(df_a['X']<0)&(df_a['Y']>0)].index #CuadranteIV: 270 a 360°
  #   df_a.loc[x0y0,"WDR"]=(round(90-((np.arctan2(df_a.loc[x0y0,'Y'],df_a.loc[x0y0,'X']))*180/np.pi),0)) #arctan2
  #   df_a.loc[xy,"WDR"]=(round(90-((np.arctan2(df_a.loc[xy,'Y'],df_a.loc[xy,'X']))*180/np.pi),0)) #arctan2
  #   df_a.loc[x0y,"WDR"]=round(270-((np.arctan(df_a.loc[x0y,'Y']/df_a.loc[x0y,'X']))*180/np.pi),0) #arctan
  #   df_a.loc[xy0,"WDR"]=round(270-((np.arctan(df_a.loc[xy0,'Y']/df_a.loc[xy0,'X']))*180/np.pi),0) #arctan

  # # Respalda base con rellemo por año-mes-hora
  # df_a.to_csv(ruta + "Sitios/" + sitio + "_rell01.csv", index=False)         

  # # visualiza el comportamiento de la base con relleno de faltantes por año-mes-hora
  # df_a.plot(subplots=True, figsize=(18,25))
  # plt.savefig(ruta + "Sitios/" + sitio + "_rell01.png", bbox_inches='tight')
  # plt.show()
  # print("No se logra el 100% de relleno de faltantes")
  # print(missing_values_table(df_a))


  # # Relleno de faltantes por año-hora con la media
  # df_a2=pd.DataFrame()
  # for y in df['year'].unique():  
  #   df_gp=df[df['year']==y]
  #   df_gp_k=df_gp.copy()
  #   h=0
  #   for h in df_gp['Hour'].unique():
  #     h_index=df_gp[df_gp['Hour']==h].index.tolist()
  #     df_gp_k.loc[h_index,:]=df_gp.loc[h_index,:].fillna(df_gp.loc[h_index,:].mean()) 
  #     df_a2=pd.concat([df_a2,df_gp_k],axis=0)

  # # Convertir X Y a WSP y WDR en df_a2
  # if df.columns.isin(["WSP"]).sum()>0:
  #   df_a2['WSP']=round(np.sqrt(df_a2['X']**2 + df_a2['X']**2),0)
  #   x0y0=df_a2[(df_a2['X']>=0)&(df_a2['Y']>=0)].index #Cuadrante I: 0 a 90°
  #   xy=df_a2[(df_a2['X']>=0)&(df_a2['Y']<0)].index #Cuadrante II: 90 a 180°
  #   x0y=df_a2[(df_a2['X']<0)&(df_a2['Y']<0)].index #Cuadrante III: 180 a 270°
  #   xy0=df_a2[(df_a2['X']<0)&(df_a2['Y']>0)].index #CuadranteIV: 270 a 360°
  #   df_a2.loc[x0y0,"WDR"]=(round(90-((np.arctan2(df_a2.loc[x0y0,'Y'],df_a2.loc[x0y0,'X']))*180/np.pi),0)) #arctan2
  #   df_a2.loc[xy,"WDR"]=(round(90-((np.arctan2(df_a2.loc[xy,'Y'],df_a2.loc[xy,'X']))*180/np.pi),0)) #arctan2
  #   df_a2.loc[x0y,"WDR"]=round(270-((np.arctan(df_a2.loc[x0y,'Y']/df_a2.loc[x0y,'X']))*180/np.pi),0) #arctan
  #   df_a2.loc[xy0,"WDR"]=round(270-((np.arctan(df_a2.loc[xy0,'Y']/df_a2.loc[xy0,'X']))*180/np.pi),0) #arctan

  # # Respalda base con relleno por año-hora
  # df_a2.to_csv(ruta + "Sitios/" + sitio + "_rell02.csv", index=False)         

  # # visualiza el comportamiento de la base con relleno de faltantes por año-hora
  # df_a2.plot(subplots=True, figsize=(18,25))
  # plt.savefig(ruta + "Sitios/" + sitio + "_rell02.png", bbox_inches='tight')
  # plt.show()
  # print("No se logra el 100% de relleno de faltantes")
  # print(missing_values_table(df_a2))




Output hidden; open in https://colab.research.google.com to view.

In [30]:
#Corrobora que las fechas se completan
days=pd.date_range(start='2015-1-1', end='2019-9-30', freq='D')
days=pd.DataFrame(days)
days.columns=['date2']

dates=pd.date_range(start='2015-1-1', end='2019-9-30', freq='H')
dates=pd.DataFrame(dates)
dates.columns=['date2']


dff=paths[0]
# Asigna el nombre del sitio
sitio=dff[-17:-14]
# Lee el archivo, cambia formato de la fecha, genera mes y elimina columnas sin datos
df = pd.read_csv(dff)
df.date2 = pd.to_datetime(df.date2, format="%Y-%m-%d %H:%M")
df=df.set_index('date2')
df.drop(columns=df.columns[df.isnull().sum()==df.shape[0]],inplace=True,axis=1)
dates[param]=np.NaN
dates=pd.DataFrame(dates)
dates=dates.set_index('date2')
days[param]=np.NaN
days=pd.DataFrame(days)
days=days.set_index('date2')
df=pd.DataFrame(df)
df=pd.merge(df, dates, on='date2',indicator='union')
df.drop(columns=df.columns[df.isnull().sum()==df.shape[0]],inplace=True,axis=1)
df.columns = [col.replace("_x","") for col in df.columns ]
print("Columnas de df ",df.columns)
print(df.head(3))
#df['Month']=df['date2'].dt.month
print("tamaño df modificado: ",df.shape[0])

df_d=df.resample('D').mean()
dates_d=dates.resample('D').mean()
print(df_d.shape)
print(dates_d.shape)
print(days.shape)

print(df_d.columns)
print(dates_d.columns)
print(days.columns)

df_d.head()

Columnas de df  Index(['date', 'Date', 'Hour', 'year', 'id_station', 'CO', 'NO', 'NO2', 'NOX',
       'O3', 'PM2.5', 'SO2', 'union'],
      dtype='object')
                                 date        Date  Hour  ...  PM2.5  SO2  union
date2                                                    ...                   
2015-01-01 00:00:00  01/01/2015 01:00  01/01/2015     0  ...  118.0  4.0   both
2015-01-01 01:00:00  01/01/2015 02:00  01/01/2015     1  ...  107.0  5.0   both
2015-01-01 02:00:00  01/01/2015 03:00  01/01/2015     2  ...  121.0  4.0   both

[3 rows x 13 columns]
tamaño df modificado:  39731
(1734, 9)
(1734, 20)
(1734, 20)
Index(['Hour', 'year', 'CO', 'NO', 'NO2', 'NOX', 'O3', 'PM2.5', 'SO2'], dtype='object')
Index(['date', 'Date', 'Hour', 'year', 'id_station', 'CO', 'NO', 'NO2', 'NOX',
       'O3', 'PA', 'PM10', 'PM2.5', 'PMCO', 'RH', 'SO2', 'TMP', 'WDR', 'WSP',
       'Month'],
      dtype='object')
Index(['date', 'Date', 'Hour', 'year', 'id_station', 'CO', 'NO', 'NO2', 'NOX

,Hour,year,CO,NO,NO2,NOX,O3,PM2.5,SO2
date2,,,,,,,,,
2015-01-01,11.5,2015.0,1.320833,13.708333,27.541667,41.250000,21.625000,67.857143,3.708333
2015-01-02,11.5,2015.0,0.966667,20.363636,21.500000,41.909091,16.458333,9.125000,0.625000
2015-01-03,11.5,2015.0,0.908333,15.125000,19.500000,34.666667,12.750000,11.260870,0.541667
2015-01-04,11.5,2015.0,0.881818,NaN,NaN,NaN,17.681818,11.500000,0.909091
2015-01-05,11.5,2015.0,0.645833,NaN,NaN,NaN,15.291667,10.391304,5.958333


In [ ]:

    # Conversión de WSP y WDR a cartesianas (otra opción es con import math math.sin, math.cos, math.radians)
    if df.columns.isin(["WSP"]).sum()>0:
      wsp=df['WSP']
      wdr=df['WDR']
      df['X']=wsp*np.sin(wdr*np.pi/180)
      df['Y']=wsp*np.cos(wdr*np.pi/180)
      
    # visualiza el comportamiento de la base original
    df.plot(subplots=True, figsize=(18,25))
    plt.savefig(ruta + "Sitios/" + sitio + "_sinrell.png", bbox_inches='tight')
    plt.show()

    # Despliega la tabla de datos faltantes y genera la lista de parametros a rellenar
    faltantes=missing_values_table(df)
    param=faltantes.index
    print('Los faltantes van de ',faltantes['% of Total Values'].min(),'% a ',faltantes['% of Total Values'].max(),'%')
    print(faltantes)
    
    # Explora el comportamiento de los faltantes
    import missingno as msno
    print(msno.bar(df[param]))
    print(msno.matrix(df[param]))  
    # Ordena por la varable de interés (PM2.5)
    sorted = df[param].sort_values('PM2.5')
    print(msno.matrix(sorted))
    msno.heatmap(df[param]).get_figure().savefig(ruta + "Sitios/" + sitio + "_sinrell_heatmap.png", bbox_inches='tight')
    msno.dendrogram(df[param]).get_figure().savefig(ruta + "/Sitios/" + sitio + "_sinrell_dendr.png", bbox_inches='tight')
    print("El comportamiento de los datos faltantes no es aleatorio ya que muestra patrones de asociación entre columnas, por un lado las partículas (PM10, PM2.5, PMCO), por otro los gases (NO, NO2, NOX, CO, SO2, O3), otro más entre los meteorológicos (TMP, HR) y (WSP, WDR), y al final PA.")

    # Relleno de faltantes por año-mes-hora con la media
    df_a=pd.DataFrame()
    for y in df['year'].unique():  
      m=0
      for m in df['Month'].unique():
        df_gp=df[(df['year']==y)&(df['Month']==m)]
        df_gp_k=df_gp.copy()
        h=0
        for h in df_gp['Hour'].unique():
            h_index=df_gp[df_gp['Hour']==h].index.tolist()
            df_gp_k.loc[h_index,:]=df_gp.loc[h_index,:].fillna(df_gp.loc[h_index,:].mean()) 
            df_a=pd.concat([df_a,df_gp_k],axis=0)

    # Convertir X Y a WSP y WDR en df_a
    if df.columns.isin(["WSP"]).sum()>0:
      df_a['WSP']=round(np.sqrt(df_a['X']**2 + df_a['X']**2),0)
      x0y0=df_a[(df_a['X']>=0)&(df_a['Y']>=0)].index #Cuadrante I: 0 a 90°
      xy=df_a[(df_a['X']>=0)&(df_a['Y']<0)].index #Cuadrante II: 90 a 180°
      x0y=df_a[(df_a['X']<0)&(df_a['Y']<0)].index #Cuadrante III: 180 a 270°
      xy0=df_a[(df_a['X']<0)&(df_a['Y']>0)].index #CuadranteIV: 270 a 360°
      df_a.loc[x0y0,"WDR"]=(round(90-((np.arctan2(df_a.loc[x0y0,'Y'],df_a.loc[x0y0,'X']))*180/np.pi),0)) #arctan2
      df_a.loc[xy,"WDR"]=(round(90-((np.arctan2(df_a.loc[xy,'Y'],df_a.loc[xy,'X']))*180/np.pi),0)) #arctan2
      df_a.loc[x0y,"WDR"]=round(270-((np.arctan(df_a.loc[x0y,'Y']/df_a.loc[x0y,'X']))*180/np.pi),0) #arctan
      df_a.loc[xy0,"WDR"]=round(270-((np.arctan(df_a.loc[xy0,'Y']/df_a.loc[xy0,'X']))*180/np.pi),0) #arctan

    # Respalda base con rellemo por año-mes-hora
    df_a.to_csv(ruta + "Sitios/" + sitio + "_rell01.csv", index=False)         

    # visualiza el comportamiento de la base con relleno de faltantes por año-mes-hora
    df_a.plot(subplots=True, figsize=(18,25))
    plt.savefig(ruta + "Sitios/" + sitio + "_rell01.png", bbox_inches='tight')
    plt.show()
    print("No se logra el 100% de relleno de faltantes")
    print(missing_values_table(df_a))


    # Relleno de faltantes por año-hora con la media
    df_a2=pd.DataFrame()
    for y in df['year'].unique():  
      df_gp=df[df['year']==y]
      df_gp_k=df_gp.copy()
      h=0
      for h in df_gp['Hour'].unique():
        h_index=df_gp[df_gp['Hour']==h].index.tolist()
        df_gp_k.loc[h_index,:]=df_gp.loc[h_index,:].fillna(df_gp.loc[h_index,:].mean()) 
        df_a2=pd.concat([df_a2,df_gp_k],axis=0)

    # Convertir X Y a WSP y WDR en df_a2
    if df.columns.isin(["WSP"]).sum()>0:
      df_a2['WSP']=round(np.sqrt(df_a2['X']**2 + df_a2['X']**2),0)
      x0y0=df_a2[(df_a2['X']>=0)&(df_a2['Y']>=0)].index #Cuadrante I: 0 a 90°
      xy=df_a2[(df_a2['X']>=0)&(df_a2['Y']<0)].index #Cuadrante II: 90 a 180°
      x0y=df_a2[(df_a2['X']<0)&(df_a2['Y']<0)].index #Cuadrante III: 180 a 270°
      xy0=df_a2[(df_a2['X']<0)&(df_a2['Y']>0)].index #CuadranteIV: 270 a 360°
      df_a2.loc[x0y0,"WDR"]=(round(90-((np.arctan2(df_a2.loc[x0y0,'Y'],df_a2.loc[x0y0,'X']))*180/np.pi),0)) #arctan2
      df_a2.loc[xy,"WDR"]=(round(90-((np.arctan2(df_a2.loc[xy,'Y'],df_a2.loc[xy,'X']))*180/np.pi),0)) #arctan2
      df_a2.loc[x0y,"WDR"]=round(270-((np.arctan(df_a2.loc[x0y,'Y']/df_a2.loc[x0y,'X']))*180/np.pi),0) #arctan
      df_a2.loc[xy0,"WDR"]=round(270-((np.arctan(df_a2.loc[xy0,'Y']/df_a2.loc[xy0,'X']))*180/np.pi),0) #arctan

    # Respalda base con relleno por año-hora
    df_a2.to_csv(ruta + "Sitios/" + sitio + "_rell02.csv", index=False)         

    # visualiza el comportamiento de la base con relleno de faltantes por año-hora
    df_a2.plot(subplots=True, figsize=(18,25))
    plt.savefig(ruta + "Sitios/" + sitio + "_rell02.png", bbox_inches='tight')
    plt.show()
    print("No se logra el 100% de relleno de faltantes")
    print(missing_values_table(df_a2))

    # Relleno de faltantes por hora con la media
    df_a0=df.copy()
    for h in df['Hour'].unique():
      h_index=df[df['Hour']==h].index.tolist()
      df_a0.loc[h_index,:]=df.loc[h_index,:].fillna(df.loc[h_index,:].mean())

    # Convertir X Y a WSP y WDR en df_a0
    if df.columns.isin(["WSP"]).sum()>0:
      df_a0['WSP']=round(np.sqrt(df_a0['X']**2 + df_a0['X']**2),0)
      x0y0=df_a0[(df_a0['X']>=0)&(df_a0['Y']>=0)].index #Cuadrante I: 0 a 90°
      xy=df_a0[(df_a0['X']>=0)&(df_a0['Y']<0)].index #Cuadrante II: 90 a 180°
      x0y=df_a0[(df_a0['X']<0)&(df_a0['Y']<0)].index #Cuadrante III: 180 a 270°
      xy0=df_a0[(df_a0['X']<0)&(df_a0['Y']>0)].index #CuadranteIV: 270 a 360°
      df_a0.loc[x0y0,"WDR"]=(round(90-((np.arctan2(df_a0.loc[x0y0,'Y'],df_a0.loc[x0y0,'X']))*180/np.pi),0)) #arctan2
      df_a0.loc[xy,"WDR"]=(round(90-((np.arctan2(df_a0.loc[xy,'Y'],df_a0.loc[xy,'X']))*180/np.pi),0)) #arctan2
      df_a0.loc[x0y,"WDR"]=round(270-((np.arctan(df_a0.loc[x0y,'Y']/df_a0.loc[x0y,'X']))*180/np.pi),0) #arctan
      df_a0.loc[xy0,"WDR"]=round(270-((np.arctan(df_a0.loc[xy0,'Y']/df_a0.loc[xy0,'X']))*180/np.pi),0) #arctan

  # Corrobora que no falta ningun día
    print(missing_values_table(df_a0.resample('D').mean()))

    # Respalda base con relleno por hora
    df_a0.to_csv(ruta + "Sitios/" + sitio + "_rell00.csv", index=False)         

    # visualiza el comportamiento de la base con relleno de faltantes por hora
    df_a0.plot(subplots=True, figsize=(18,25))
    plt.savefig(ruta + "Sitios/" + sitio + "_rell00.png", bbox_inches='tight')
    plt.show()
    print("Se logra el 100% de relleno de faltantes")
    print(missing_values_table(df_a0))



#FIN Etapa 02 Relleno de Faltantes